# GMM

This notebook shows a simple example of how GMM can be used to estimate model parameters. It starts with an exactly identified case and then moves on to different ways of estimating an overidentified case (pre-defined weighting matrix, recombining the moment conditions, optimal weighting matrix).

## Load Packages and Extra Functions

In [1]:
using Printf, DelimitedFiles, Statistics, LinearAlgebra, Optim, NLsolve

include("jlFiles/printmat.jl")
include("jlFiles/CovNWFn.jl");      #Newey-West covariance matrix

# Loading Data

In [2]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)   #start on line 2, column 1
x = x[:,2]         #excess market returns, in %

T = size(x,1)

388

# GMM I

This section describes the basic (exactly identified) GMM, that is, when we have as many moment conditions as parameters. In this case GMM is the same as the classical method of moments.

## Traditional Estimation of Mean and Variance

The next cell applies the traditional way of estimating the mean and the variance. The standard errors are text book formulas.

In [3]:
μ  = mean(x)
σ² = var(x,corrected=false)       #"false" to use 1/T formula

par_a = [μ;σ²]

printblue("Traditional estimates:\n")
xx = [par_a [sqrt((σ²/T));sqrt(2*σ²^2/T)]]
colNames = ["coef","std"]
parNames = ["μ","σ²"]
printmat(xx;colNames,rowNames=parNames)      # ; since keywords with same name

Traditional estimates:

        coef       std
μ      0.602     0.233
σ²    21.142     1.518



## GMM Point Estimates

To estimate the mean and variance of $x_{t}$, use the following moment condition

$
\frac{1}{T}\sum\nolimits_{t=1}^{T}g_{t}=0 \: \text{ where } 
$

$
g_{t}(\mu,\sigma^{2})=
\begin{bmatrix}
x_{t}-\mu\\
(x_{t}-\mu)^{2}-\sigma^{2}
\end{bmatrix}.
$

The parameter values ($\mu,\sigma^2$) that make these moment conditions hold are the same as from the traditional method. It is easy to solve for these parameters when the moment conditions are linear in the parameters (as they are here). However, to facilitate adapting the code to non-linear models, we solve for the parameters by a numerical method.

### A Remark on the Code

- `Sol = nlsolve(p->Gmm2MomFn(p,x)[2],par_a)` solves for the vector `p` that makes the 2nd output of `Gmm2MomFn(p,x)==[0,0]`. The `p->Gmm2MomFn(p,x)[2]` defines an anonymous function (in terms of the vector of parameters `p`) that returns (a vector of) the sample averages of the moment conditions.
- To extract the solution, use ` Sol.zero`.

In [4]:
"""
    Gmm2MomFn(par,x)

Calculate traditional 2 moment conditions for estimating [μ,σ²]

# Input
- `par::Vector`: [μ,σ²]
- `x::Vector`:   T-vector with data

# Output
- `g::Matrix`:    Tx2, moment conditions
- `gbar::Vector`: 2-vector, means of each column in g

"""
function Gmm2MomFn(par,x)
    (μ,σ²) = (par[1],par[2])
    g      = [(x .- μ) ((x .- μ).^2 .- σ²)]  #Tx2
    gbar   = vec(mean(g,dims=1))             #2-element vector
    return g,gbar
end

Gmm2MomFn

In [5]:
Sol = nlsolve(p->Gmm2MomFn(p,x)[2],par_a)   #numerically solve for the estimates
par_1 = Sol.zero

printblue("GMM estimates:")
printmat(par_1,rowNames=parNames)

(g,gbar) = Gmm2MomFn(par_1,x)           #Tx2, moment conditions
printblue("Checking if mean of moment conditions = 0")
printmat(gbar,rowNames=["g₁","g₂"])

GMM estimates:
μ      0.602
σ²    21.142

Checking if mean of moment conditions = 0
g₁     0.000
g₂     0.000



## GMM Distribution

The distribution of the basic GMM estimates is

$
\sqrt{T}(\hat{\mu}-\mu_{0})\overset{d}{\rightarrow}N(0,V) 
\: \text{ where } \: 
V = (D_{0}^{\prime}S_{0}^{-1}D_{0})  ^{-1}
$

(This holds for exactly identified models.)

Clearly, $D_{0}=-\textrm{I}$ and if data is iid then $S_{0}=\text{Var}(g_{t})$.

In [6]:
D  = -I(2)                   #Jacobian, does not really matter here
S  = CovNWFn(g,1)/T          #Newey-West with 1 lag
V1 = inv(D'inv(S)*D)

printblue("GMM estimates:\n")
xx = [par_1 sqrt.(diag(V1/T))]
printmat(xx;colNames,rowNames=parNames)

printstyled("Compare with the traditional estimates",color=:red,bold=true)

GMM estimates:

        coef       std
μ      0.602     0.244
σ²    21.142     2.381

Compare with the traditional estimates

# GMM II

This section expands the GMM calculations to handle an overidentified case: more moment conditions than parameters.

Warning: some of the variables (`g,S`, etc) are overwritten with new values.

## The Moment Conditions

If $x_{t}$ is $N(\mu,\sigma^{2})$, then the following moment conditions should
all be zero (in expectation)

$
g_{t}(\mu,\sigma^{2})=
\begin{bmatrix}
x_{t}-\mu\\
(x_{t}-\mu)^{2}-\sigma^{2}\\
(x_{t}-\mu)^{3}\\
(x_{t}-\mu)^{4}-3\sigma^{4}
\end{bmatrix}.
$

The first moment condition defines the mean $\mu$, the second defines the
variance $\sigma^{2}$, while the third and forth are the skewness and excess
kurtosis respectively.

The next cell also has a function for calculating the jacobian of the moment conditions. In this linear model, it is fairly straightforward to code. Otherwise, we could apply a numerical algorithm for calculating derivatives, for instance, from the [FiniteDiff.jl](https://github.com/JuliaDiff/FiniteDiff.jl) package.

In [7]:
"""
    Gmm4MomFn(par,x)

Calculate 4 moment conditions for estimating [μ,σ²]

# Input
- `par::Vector`: [μ,σ²]
- `x::Vector`:   T-vector with data

"""
function Gmm4MomFn(par,x)
  (μ,σ²) = (par[1],par[2])
  g      = [(x .- μ) ((x .- μ).^2 .- σ²) ((x .- μ).^3) ((x .- μ).^4 .- 3*σ²^2)]    #Tx4
  gbar   = vec(mean(g,dims=1))     #4-element vector
  return g,gbar
end

"""
    DGmm4MomFn(par,x)

Calculate (analytical) Jacobian of Gmm4MomFn(), 4x2

"""
function DGmm4MomFn(par,x)
    (μ,σ²) = (par[1],par[2])
    D  = [-1                  0    ;     #Jacobian of Gmm4MomFn, 4x2
          -2*mean(x.-μ)      -1    ;
          -3*mean((x.-μ).^2)   0   ;
          -4*mean((x.-μ).^3)  -6*σ²]
    return D
end

DGmm4MomFn

## GMM: Minimizing gbar'W*gbar


The following code applies a numerical method to solve a minimization problem with the weighting matrix 

$
W=
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0
\end{bmatrix}
$

The results should be the same (or at least very close to) the previous results, since the $W$ matrix puts all weight on the first two moments (basically mimicking the estimations above). Changing $W$, for instance, by setting $W[3,3]=0.0001$ will give other estimates.

We define the loss function as $\bar{g}'W\bar{g}$.

As a practical matter, it is often the case that a derivative-free method works better than other optimization routines.

The expressions for variance-covatriance matrix are in the lecture notes.

### A Remark on the Code

- `diagm(0=>[1.0,1.0,0.0,0.0])` creates a $W$ matrix like the one above.

- `optimize(par->Gmm4MomLossFn(par,x,W),par_a)` find the vetcor `par` that minimzes `Gmm4MomLossFn()`.


In [8]:
"""
    Gmm4MomLossFn(par,x,W=1)

Calculate loss function from moment conditions in Gmm4MomFn() and a weighting matrix W.

# Input
-`par`: see Gmm4MomFn 
-`x`:   see Gmm4MomFn
-`W::Number or Matrix`:  weighting matrix (or just any positive number)

# Output
- `Loss:: Number`: Loss function value

"""
function Gmm4MomLossFn(par,x,W=1)
  (g,gbar) = Gmm4MomFn(par,x)
  Loss     = gbar'W*gbar      #to be minimized
  return Loss
end

Gmm4MomLossFn

In [9]:
W     = diagm(0=>[1.0,1.0,0.0,0.0])   #weighting matrix, try changing it
#W[3,3] = 0.0001
Sol   = optimize(par->Gmm4MomLossFn(par,x,W),par_a)
par_2 = Optim.minimizer(Sol)

printblue("GMM estimates from minimizing gbar'W*gbar:")
printmat(par_2,rowNames=parNames)

GMM estimates from minimizing gbar'W*gbar:
μ      0.602
σ²    21.142



In [10]:
momNames = ["g₁","g₂","g₃","g₄"]

D  = DGmm4MomFn(par_2,x)               #Jacobian, 4x2
printblue("The Jacobian is:\n")
printmat(D,rowNames=momNames,colNames=parNames)

g,    = Gmm4MomFn(par_2,x)                #Tx4, moment conditions, evaluated at point estimates
S     = CovNWFn(g,1)/T                    #variance of sqrt(T)"gbar, NW with 1 lag
V2    = inv(D'W*D)*D'W*S*W'D*inv(D'W*D)   #see lecture notes for V2

printblue("Weighting matrix:\n")
printmat(W,colNames=momNames,rowNames=momNames)

printblue("GMM estimates (gbar'W*gbar):\n")
xx = [par_2 sqrt.(diag(V2/T))]
printmat(xx;colNames,rowNames=parNames)

The Jacobian is:

           μ        σ²
g₁    -1.000     0.000
g₂    -0.000    -1.000
g₃   -63.427     0.000
g₄   314.797  -126.854

Weighting matrix:

          g₁        g₂        g₃        g₄
g₁     1.000     0.000     0.000     0.000
g₂     0.000     1.000     0.000     0.000
g₃     0.000     0.000     0.000     0.000
g₄     0.000     0.000     0.000     0.000

GMM estimates (gbar'W*gbar):

        coef       std
μ      0.602     0.244
σ²    21.142     2.381



## Checking the Calculation of the Jacobian (extra)

by using also a numerical method from the [FiniteDiff.jl](https://github.com/JuliaDiff/FiniteDiff.jl) package.

### A Remark on the Code

- We rename `finite_difference_jacobian` as `jacobian` (to make it more convenient to use).

In [11]:
using FiniteDiff: finite_difference_jacobian as jacobian      

D_num = jacobian(par->Gmm4MomFn(par,x)[2],par_2)

printblue("The numerical Jacobian is:\n")
printmat(D_num,rowNames=momNames,colNames=parNames)

printred("Compare with the results from DGmm4MomFn(), see above")

The numerical Jacobian is:

           μ        σ²
g₁    -1.000     0.000
g₂    -0.000    -1.000
g₃   -63.427     0.000
g₄   314.797  -126.854

Compare with the results from DGmm4MomFn(), see above


# GMM: Minimizing gbar'W*gbar, Iterating over W


The following code iterates over the weighting matrix by using $W=S^{-1}$, where

$S = \text{Cov}(\sqrt{T}\bar{g})$

is from the previous iteration.

### A Remark on the Code

- `(maximum(abs,par - par_old) > 1e-3) || (i < 2)` loops while the |change| in the parameters > 1e-3 (and at least once).

- `copyto!(par_old,par)` overwrites `par_old` with the contents in `par`

In [12]:
println("\niterated GMM, using optimal weighting matrix, starting with S from previous estimation")

(par,par_old,S,i) = (copy(par_1),fill(Inf,length(par_1)),copy(S),1)

println("\n\niterating over W starting with the W choice above")
while (maximum(abs,par - par_old) > 1e-3) || (i < 2)    #require at least one iteration
    #global par, par_old, i, W, S    #only needed in script
    local Sol, g
    println("-------iteration  $i, old and new parameters--------")
    copyto!(par_old,par)          #update par_old            
    W               = inv(S)
    Sol             = optimize(par->Gmm4MomLossFn(par,x,W),par_old)
    par             = Optim.minimizer(Sol)
    printlnPs(par_old')
    printlnPs(par')
    g,              = Gmm4MomFn(par,x)
    S               = CovNWFn(g,1)/T
    i               = i + 1
 end

V2 = inv(D'W*D)*D'W*S*W'D*inv(D'W*D)      #if non-optimal weighting matrix
V1 = inv(D'inv(S)*D)                      #with optimal weighting matrix

printblue("\nGMM estimates (gbar'W*gbar, iteration over W):")
xx = [par sqrt.(diag(V2/T)) sqrt.(diag(V1/T))]
printmat(xx,colNames=[colNames;"std ver. 2"],rowNames=parNames,width=12)


iterated GMM, using optimal weighting matrix, starting with S from previous estimation


iterating over W starting with the W choice above
-------iteration  1, old and new parameters--------
     0.602    21.142
     0.877    16.916
-------iteration  2, old and new parameters--------
     0.877    16.916
     0.879    16.648
-------iteration  3, old and new parameters--------
     0.879    16.648
     0.879    16.645
-------iteration  4, old and new parameters--------
     0.879    16.645
     0.879    16.647
-------iteration  5, old and new parameters--------
     0.879    16.647
     0.879    16.647

GMM estimates (gbar'W*gbar, iteration over W):
          coef         std  std ver. 2
μ        0.879       0.217       0.217
σ²      16.647       1.311       1.311



In [13]:
printblue("W matrix used in the last iteration, (times 10000):\n")

printmat(W*10000,colNames=momNames,rowNames=momNames)

W matrix used in the last iteration, (times 10000):

          g₁        g₂        g₃        g₄
g₁  1525.564    39.433   -16.963    -0.674
g₂    39.433    18.778    -0.297    -0.050
g₃   -16.963    -0.297     0.306     0.012
g₄    -0.674    -0.050     0.012     0.001



## GMM: A*g = 0


The following code from estimates the parameters by combining the 4 original moment conditions in $\bar{g}$ into 2
effective moment conditions, $A\bar{g}$, where $A$ is a $2\times4$ matrix

$
A=
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0
\end{bmatrix}
$

This particular $A$ matrix implies that we use the classical
estimators of the mean and variance. Changing $A$, for instance, by setting $A[1,3]=0.001$ will give different estimates.

In [14]:
A = [1 0 0 0;                   #A in A*gbar=0 (here: all weight on first two moments)
     0 1 0 0]                   #try setting A[1,3] = 0.001

Sol   = nlsolve(p->A*Gmm4MomFn(p,x)[2],par_a)   #solve for the GMM estimates
par_3 = Sol.zero

printblue("GMM estimates from A*gbar=0:")
printmat(par_3,rowNames=parNames)

(g,gbar) = Gmm4MomFn(par_3,x)        #Tx4, moment conditions. Warning: overwriting old g
q = size(g,2)

printblue("\nChecking if mean of A*g_t = 0")
printmat(A*gbar,rowNames=["A₁g","A₂g"])

GMM estimates from A*gbar=0:
μ      0.602
σ²    21.142


Checking if mean of A*g_t = 0
A₁g     0.000
A₂g     0.000



In [15]:
D  = DGmm4MomFn(par_3,x)               #Jacobian
printblue("The Jacobian is:")
printmat(D,colNames=parNames,rowNames=momNames)

S  = CovNWFn(g,1)/T
V3 = inv(A*D)*A*S*A'inv(A*D)'          #see lecture notes for V3

printblue("GMM estimates (A*gbar):\n")
xx = [par_3 sqrt.(diag(V3/T))]
printmat(xx;colNames,rowNames=parNames)

printstyled("Compare with the exactly identified GMM (above)",color=:red,bold=true)

The Jacobian is:
           μ        σ²
g₁    -1.000     0.000
g₂    -0.000    -1.000
g₃   -63.427     0.000
g₄   314.797  -126.854

GMM estimates (A*gbar):

        coef       std
μ      0.602     0.244
σ²    21.142     2.381

Compare with the exactly identified GMM (above)